<a href="https://colab.research.google.com/github/Sunnnyyy16/EmoSupportAI/blob/main/MH_LLaMa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MH-LLaMa


*   LLaMa3.2-1B 모델을 Mental-health conversation 데이터셋으로 학습
*   Low-Rank Adaptation (LoRA)



In [ ]:
!pip install datasets jsonlines

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
!pip install datasets jsonlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd

import json
import jsonlines

from datasets import Dataset

In [ ]:
import csv
import json

def csv_to_jsonl(csv_file_path, jsonl_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
            for row in csv_reader:
                json_line = json.dumps(row)
                jsonl_file.write(json_line + '\n')

input_file = 'MentalWell.csv'
output_file = 'MentalWell.jsonl'

try:
    csv_to_jsonl(input_file, output_file)
    print(f"Successfully converted {input_file} to {output_file}")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Successfully converted MentalWell.csv to MentalWell.jsonl


In [ ]:
datasetName = "MentalWell.csv"
jsonFileName = "MentalWell.jsonl"

In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    df = pd.read_csv(csv_file_path)

    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        for index, row in df.iterrows():
            data = {'inputs': row['questionText'], 'response': row['answerText']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')

csv_file_path = datasetName
json_file_path = jsonFileName

csv_to_json(csv_file_path, json_file_path)

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.16.1
    Uninstalling datasets-2.16.1:
      Successfully uninstalled datasets-2.16.1


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
base_model = "meta-llama/Llama-3.2-1B"

code_dataset = "dhlim55/mental_health"

new_model = "llama-3.2-1b-formentalhealth"

In [ ]:
dataset = load_dataset(code_dataset, split="train")

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/274 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.57M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3508 [00:00<?, ? examples/s]

In [ ]:
print( dataset[28] )

{'text': '<s>[INST] I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling? [/INST] This is a great question! I personally don\'t believe that any client could ever have too many issues for counseling. In fact, that type of thinking may be stopping you from seeking counseling, so it may be hindering you from getting the help you need. In fact, all of what you described points to the importance of you seeking help in order to cope with the many challenges in your life.\xa0If you seek counseling, it will be important for you to understand that you may need to remain in counseling for a sustained period of time in order to work through each of these issues. All of these iss

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3508 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,2.673500
50,2.609700
75,2.470900
100,2.583200
125,2.510600
150,2.536500
175,2.455200
200,2.466400
225,2.420000
250,2.472500


TrainOutput(global_step=5262, training_loss=2.1907767433940752, metrics={'train_runtime': 1427.754, 'train_samples_per_second': 7.371, 'train_steps_per_second': 3.686, 'total_flos': 1.815802681761792e+16, 'train_loss': 2.1907767433940752, 'epoch': 3.0})

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Hello, I’m seeking advice about a recent concern regarding a friendship. \
My best friend has started dating someone, but I don’t have a good impression of her boyfriend.\
 He seems to have a violent temperament and often speaks harshly to her. Last week, the three of us met up, and he even shouted at her in a public cafe.\
  I want to tell my friend that it would be better for her not to see him, but I’m afraid this might strain our friendship.\
   What is the best course of action I can take in this situation?"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")

print(result[0]['generated_text'])


<s>[INST] Hello, I’m seeking advice about a recent concern regarding a friendship. My best friend has started dating someone, but I don’t have a good impression of her boyfriend. He seems to have a violent temperament and often speaks harshly to her. Last week, the three of us met up, and he even shouted at her in a public café. I want to tell my friend that it would be better for her not to see him, but I’m afraid this might strain our friendship. What is the best course of action I can take in this situation? [/INST] Hi, I'm sorry you feel like you are not on the same page with your friend. I would suggest that you talk to your friend about your concerns. You can say something like "I feel like I am not on the same page with you and I want to be clear about what I am saying. I want to let you know that I don't like what you're doing and I think it would be better if you didn't see him. I don't want to lose our friendship, but I think it would be better for all of us if we didn't see 